In [1]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

In [2]:
from keras.utils.np_utils import to_categorical

X_train = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\x_train_encode.csv")
X_test = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\x_test_encode.csv")
y_train = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\y_train_encode.csv")
y_test = pd.read_csv(r"D:\Northeastern Semester 1\Projects\ml_project\Data_Set\y_test_encode.csv")

X_train.drop(columns = ['Unnamed: 0'], axis =1, inplace = True)
X_test.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)

""" y_train = to_categorical(y_train)
y_test =  to_categorical(y_test) """
y_test.head()

,Y
0,0
1,1
2,0
3,1
4,0


In [3]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv, scoring,
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)

    pred = gs.predict(X_test_data, y_test_data)

    return fitted_model, pred

In [4]:
def modell(activation, dropout,
              optimizer):
    
    
    model = Sequential()
    
    model.add(Dense(200,activation))
    model.add(Dropout(dropout))
    model.add(Dense(60,activation))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizer,
    metrics=['accuracy'])
    
    return model

tf = modell(activation= 'sigmoid', optimizer= 'Adam', dropout= 0.2)

tf.fit(X_train, y_train, batch_size= 128, epochs= 300)

tf.evaluate(X_test, y_test)

Epoch 1/300
70/70 [==============================] - 1s 3ms/step - loss: 0.6910 - accuracy: 0.5564
Epoch 2/300
70/70 [==============================] - 0s 4ms/step - loss: 0.6525 - accuracy: 0.6219
Epoch 3/300
70/70 [==============================] - 0s 3ms/step - loss: 0.6291 - accuracy: 0.6501
Epoch 4/300
70/70 [==============================] - 0s 3ms/step - loss: 0.6162 - accuracy: 0.6637
Epoch 5/300
70/70 [==============================] - 0s 4ms/step - loss: 0.6128 - accuracy: 0.6659
Epoch 6/300
70/70 [==============================] - 0s 3ms/step - loss: 0.6102 - accuracy: 0.6700
Epoch 7/300
70/70 [==============================] - 0s 4ms/step - loss: 0.6053 - accuracy: 0.6767
Epoch 8/300
70/70 [==============================] - 0s 3ms/step - loss: 0.6031 - accuracy: 0.6808
Epoch 9/300
70/70 [==============================] - 0s 3ms/step - loss: 0.6020 - accuracy: 0.6785
Epoch 10/300
70/70 [==============================] - 0s 4ms/step - loss: 0.6005 - accuracy: 0.6825
Epoch 11/

[0.6288939714431763, 0.7306883931159973]

In [5]:
param_grid = {
              'epochs':[100, 200, 200, 500],
              'batch_size':[64, 128, 256],
              'optimizer' :['Adam', 'Nadam'],
              'dropout' : [0.2,0.3, 0.5],
              'activation' : ['relu', 'elu']
             }

model = KerasClassifier(build_fn = modell, verbose=0)

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                        param_grid, cv=10, scoring='accuracy')

print(model.best_score_)
print(model.best_params_)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


c:\Users\Hemant\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  if __name__ == '__main__':
